In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import os
import matplotlib.pyplot as plt
from pathlib import Path

# ================= CONFIGURATION =================
base_path = r"D:\impress_project\eeg_signals\data\LRMI-21679035\organized_data_v2"
raw_data_dir = os.path.join(base_path, "raw_data")
output_dir = os.path.join(base_path, "raw_labelled_results")

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# ================= CHANNEL INFORMATION FROM PAPER =================
# From paper Table 2 (0-based indexing for Python)
paper_channels = {
    0: 'Fp1', 1: 'Fp2', 2: 'Pz', 3: 'P3', 4: 'P4', 5: 'F7', 6: 'F8', 7: 'FC2', 
    8: 'FC3', 9: 'FC4', 10: 'FT7', 11: 'FT8', 12: 'Cz', 13: 'C3', 14: 'C4',
    15: 'T3', 16: 'T4', 17: 'CPz', 18: 'CP3', 19: 'CP4', 20: 'TP7', 21: 'TP8',
    22: 'Pz', 23: 'P3', 24: 'P4', 25: 'T5', 26: 'T6', 27: 'Oz', 28: 'O1',
    29: 'O2', 30: 'HEOL', 31: 'VEOR', 32: 'Marker'
}

# Classify channels
channels_all = [paper_channels[i] for i in range(33)]
channels_eeg = [paper_channels[i] for i in range(30)]  # First 30 are EEG
channels_eog = [paper_channels[30], paper_channels[31]]  # HEOL, VEOR
channels_marker = [paper_channels[32]]  # Marker channel

print(f"Channels All: {len(channels_all)}")
print(f"Channels EEG: {len(channels_eeg)}")
print(f"Channels EOG: {len(channels_eog)}")
print(f"Channels Marker: {len(channels_marker)}")

# ================= FUNCTION TO PROCESS ONE SUBJECT =================
def process_raw_mat_file(subject_id, raw_data_dir, output_dir, sampling_rate=500):
    """
    Process raw .mat file for one subject and save in the specified format
    
    Parameters:
    -----------
    subject_id : str
        Subject ID (e.g., '01', '02')
    raw_data_dir : str
        Directory containing raw .mat files
    output_dir : str
        Output directory for saved results
    sampling_rate : int
        Sampling rate in Hz (500 from paper)
    
    Returns:
    --------
    result_dict : dict
        Dictionary with all required keys
    """
    
    print(f"\nProcessing subject: {subject_id}")
    
    # Construct file path
    mat_filename = f"sub-{subject_id}_task-motor-imagery_eeg.mat"
    mat_path = os.path.join(raw_data_dir, mat_filename)
    
    if not os.path.exists(mat_path):
        print(f"  ❌ File not found: {mat_path}")
        return None
    
    try:
        # Load .mat file
        data = loadmat(mat_path)
        eeg_struct = data['eeg'][0, 0]
        
        # Extract data and labels
        rawdata = eeg_struct['rawdata']  # Shape: (40, 33, 4000)
        labels = eeg_struct['label'].flatten()  # Shape: (40,)
        
        print(f"  ✓ Loaded data: {rawdata.shape}")
        print(f"  ✓ Labels: {labels.shape}")
        
        # Verify dimensions
        if rawdata.shape != (40, 33, 4000):
            print(f"  ⚠️ Unexpected shape: {rawdata.shape}, expected (40, 33, 4000)")
        
        # Separate all channels data (40 trials × 33 channels × 4000 samples)
        segments_all = rawdata  # Already in correct shape
        
        # Extract only EEG channels (first 30 channels)
        segments_eeg = rawdata[:, :30, :]  # 40 trials × 30 EEG channels × 4000 samples
        
        print(f"  ✓ Segments all: {segments_all.shape}")
        print(f"  ✓ Segments EEG: {segments_eeg.shape}")
        
        # Create event_info DataFrame
        # Each trial is 4 seconds MI period (from paper)
        event_info = []
        for trial_idx in range(40):
            label = labels[trial_idx]
            hand = "left" if label == 1 else "right"
            
            event_info.append({
                'trial_idx': trial_idx,
                'onset_sec': trial_idx * 8.0,  # Each trial starts 8s apart (2s instr + 4s MI + 2s break)
                'duration_sec': 4.0,  # MI duration
                'label': label,
                'hand': hand,
                'condition': 'motor_imagery'
            })
        
        event_info_df = pd.DataFrame(event_info)
        
        # Create result dictionary
        result_dict = {
            'segments_all': segments_all.astype(np.float32),  # Reduce memory usage
            'segments_eeg': segments_eeg.astype(np.float32),
            'labels': labels.astype(np.int32),
            'channels_all': channels_all,
            'channels_eeg': channels_eeg,
            'channels_eog': channels_eog,
            'channels_marker': channels_marker,
            'sampling_rate': sampling_rate,
            'subject_id': f"sub-{subject_id}",
            'event_info': event_info_df.to_dict('records')  # Save as list of dicts for JSON compatibility
        }
        
        # Save to .npz file
        output_path = os.path.join(output_dir, f"sub-{subject_id}_raw_labelled.npz")
        np.savez_compressed(output_path, **result_dict)
        
        print(f"  ✅ Saved to: {output_path}")
        
        # Print summary
        left_count = np.sum(labels == 1)
        right_count = np.sum(labels == 2)
        print(f"  📊 Summary: {left_count} left hand, {right_count} right hand trials")
        
        return result_dict
        
    except Exception as e:
        print(f"  ❌ Error processing {subject_id}: {e}")
        return None

# ================= PROCESS ALL SUBJECTS =================
def process_all_subjects(raw_data_dir, output_dir, max_subjects=None):
    """
    Process all subjects in the raw_data directory
    
    Parameters:
    -----------
    raw_data_dir : str
        Directory containing raw .mat files
    output_dir : str
        Output directory for saved results
    max_subjects : int or None
        Maximum number of subjects to process (for testing)
    """
    
    # Get all .mat files
    mat_files = [f for f in os.listdir(raw_data_dir) if f.endswith('.mat')]
    
    # Sort by subject number
    mat_files.sort(key=lambda x: int(x.split('-')[1].split('_')[0]))
    
    if max_subjects:
        mat_files = mat_files[:max_subjects]
    
    print(f"Found {len(mat_files)} raw .mat files")
    print(f"Output directory: {output_dir}")
    
    all_results = {}
    
    for mat_file in mat_files:
        # Extract subject ID
        subject_id = mat_file.split('-')[1].split('_')[0]  # Get '01' from 'sub-01_task-...'
        
        # Process this subject
        result = process_raw_mat_file(subject_id, raw_data_dir, output_dir)
        
        if result:
            all_results[subject_id] = result
    
    # Create summary statistics
    if all_results:
        create_summary_statistics(all_results, output_dir)
    
    return all_results

# ================= CREATE SUMMARY STATISTICS =================
def create_summary_statistics(all_results, output_dir):
    """
    Create summary CSV file for all processed subjects
    """
    summary_data = []
    
    for subject_id, result in all_results.items():
        labels = result['labels']
        left_count = np.sum(labels == 1)
        right_count = np.sum(labels == 2)
        
        summary_data.append({
            'subject_id': result['subject_id'],
            'total_trials': len(labels),
            'left_hand_trials': left_count,
            'right_hand_trials': right_count,
            'sampling_rate': result['sampling_rate'],
            'channels_eeg': len(result['channels_eeg']),
            'channels_eog': len(result['channels_eog']),
            'channels_marker': len(result['channels_marker'])
        })
    
    # Create DataFrame and save
    summary_df = pd.DataFrame(summary_data)
    summary_path = os.path.join(output_dir, "summary_statistics.csv")
    summary_df.to_csv(summary_path, index=False)
    
    print(f"\n📊 Summary saved to: {summary_path}")
    print(summary_df)
    
    # Also save a combined dataset (optional)
    save_combined_dataset(all_results, output_dir)

# ================= SAVE COMBINED DATASET =================
def save_combined_dataset(all_results, output_dir):
    """
    Save a combined dataset with all subjects (EEG only for ML)
    """
    all_segments_eeg = []
    all_labels = []
    all_subject_ids = []
    
    for subject_id, result in all_results.items():
        all_segments_eeg.append(result['segments_eeg'])
        all_labels.append(result['labels'])
        all_subject_ids.extend([result['subject_id']] * len(result['labels']))
    
    # Combine all data
    all_segments_eeg = np.vstack(all_segments_eeg)
    all_labels = np.concatenate(all_labels)
    
    # Save combined dataset
    combined_path = os.path.join(output_dir, "combined_raw_dataset.npz")
    np.savez_compressed(
        combined_path,
        segments_eeg=all_segments_eeg,
        labels=all_labels,
        subject_ids=np.array(all_subject_ids),
        channels_eeg=all_results[list(all_results.keys())[0]]['channels_eeg'],
        sampling_rate=all_results[list(all_results.keys())[0]]['sampling_rate'],
        n_subjects=len(all_results),
        n_trials=len(all_segments_eeg)
    )
    
    print(f"\n💾 Combined dataset saved to: {combined_path}")
    print(f"   Shape: {all_segments_eeg.shape}")
    print(f"   Subjects: {len(all_results)}")
    print(f"   Total trials: {len(all_segments_eeg)}")

# ================= VERIFICATION FUNCTION =================
def verify_saved_file(subject_id, output_dir):
    """
    Verify that the saved file contains all required keys
    """
    file_path = os.path.join(output_dir, f"sub-{subject_id}_raw_labelled.npz")
    
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return False
    
    try:
        # Load the file
        data = np.load(file_path, allow_pickle=True)
        
        print(f"\nVerifying {os.path.basename(file_path)}:")
        print("-" * 50)
        
        # Check all required keys
        required_keys = [
            'segments_all', 'segments_eeg', 'labels', 'channels_all',
            'channels_eeg', 'channels_eog', 'channels_marker',
            'sampling_rate', 'subject_id', 'event_info'
        ]
        
        all_present = True
        for key in required_keys:
            if key in data:
                value = data[key]
                if hasattr(value, 'shape'):
                    print(f"✓ {key}: shape={value.shape}, dtype={value.dtype}")
                else:
                    print(f"✓ {key}: {type(value)}")
            else:
                print(f"✗ {key}: MISSING!")
                all_present = False
        
        # Print sample information
        if 'labels' in data:
            labels = data['labels']
            print(f"\nLabel distribution:")
            print(f"  Left hand (1): {np.sum(labels == 1)}")
            print(f"  Right hand (2): {np.sum(labels == 2)}")
        
        if 'segments_all' in data:
            segments = data['segments_all']
            print(f"\nSegment information:")
            print(f"  Shape: {segments.shape}")
            print(f"  Memory: {segments.nbytes / 1024 / 1024:.2f} MB")
        
        data.close()
        return all_present
        
    except Exception as e:
        print(f"Error verifying file: {e}")
        return False

# ================= MAIN EXECUTION =================
if __name__ == "__main__":
    print("=" * 60)
    print("PROCESSING RAW .MAT FILES")
    print("=" * 60)
    
    # Test with first 2 subjects
    print("\nTesting with first 2 subjects...")
    test_results = process_all_subjects(raw_data_dir, output_dir, max_subjects=2)
    
    # Verify the saved files
    if test_results:
        print("\n" + "=" * 60)
        print("VERIFYING SAVED FILES")
        print("=" * 60)
        
        for subject_id in list(test_results.keys())[:2]:
            verify_saved_file(subject_id, output_dir)
    
    # Ask if user wants to process all subjects
    print("\n" + "=" * 60)
    response = input("Process ALL subjects? (y/n): ")
    
    if response.lower() == 'y':
        print("\nProcessing all subjects...")
        all_results = process_all_subjects(raw_data_dir, output_dir)
        
        print("\n" + "=" * 60)
        print("PROCESSING COMPLETE!")
        print("=" * 60)
        print(f"Results saved in: {output_dir}")
        print(f"Files created:")
        
        # List created files
        created_files = os.listdir(output_dir)
        for file in sorted(created_files):
            if file.endswith('.npz') or file.endswith('.csv'):
                file_path = os.path.join(output_dir, file)
                file_size = os.path.getsize(file_path) / 1024 / 1024
                print(f"  {file} ({file_size:.2f} MB)")
    
    else:
        print("Processing stopped. Only test files were created.")



Channels All: 33
Channels EEG: 30
Channels EOG: 2
Channels Marker: 1
PROCESSING RAW .MAT FILES

Testing with first 2 subjects...
Found 2 raw .mat files
Output directory: D:\impress_project\eeg_signals\data\LRMI-21679035\organized_data_v2\raw_labelled_results

Processing subject: 01
  ✓ Loaded data: (40, 33, 4000)
  ✓ Labels: (40,)
  ✓ Segments all: (40, 33, 4000)
  ✓ Segments EEG: (40, 30, 4000)
  ✅ Saved to: D:\impress_project\eeg_signals\data\LRMI-21679035\organized_data_v2\raw_labelled_results\sub-01_raw_labelled.npz
  📊 Summary: 20 left hand, 20 right hand trials

Processing subject: 02
  ✓ Loaded data: (40, 33, 4000)
  ✓ Labels: (40,)
  ✓ Segments all: (40, 33, 4000)
  ✓ Segments EEG: (40, 30, 4000)
  ✅ Saved to: D:\impress_project\eeg_signals\data\LRMI-21679035\organized_data_v2\raw_labelled_results\sub-02_raw_labelled.npz
  📊 Summary: 20 left hand, 20 right hand trials

📊 Summary saved to: D:\impress_project\eeg_signals\data\LRMI-21679035\organized_data_v2\raw_labelled_results\s